<a href="https://colab.research.google.com/github/Techie-Sab/machinelearning/blob/main/Hugging_Face_QNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)

Mounted at /content/drive


In [3]:
!pip install  transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.2 MB/s 
     |████████████████████████████████| 596 kB 36.8 MB/s 
     |████████████████████████████████| 6.6 MB 34.8 MB/s 
     |████████████████████████████████| 86 kB 4.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
# Adding imports

import pandas as pd
import numpy as np
import os
import re
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import pipeline
import warnings
warnings.filterwarnings('ignore')

In [5]:
class QNA():
    def __init__(self):

      try:

        self.tokenizer=AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
        self.model= AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
        self.qa=pipeline('question-answering', model=self.model, tokenizer=self.tokenizer)
      except Exception as e:
        print(str(e))

    def cleaning_text(self,text):
      text = text.lstrip()
      text = text.rstrip()
      self.temp = re.sub("\n+"," ",text)
    
      return self.temp

    def save_model(self):
      #saving the tokens
      self.tokenizer.save_pretrained('/content/drive/MyDrive/tokenizer/')
      #saving the model
      self.model.save_pretrained('/content/drive/MyDrive/Q&A/')


    def predict_ans(self,context,ques):
      ans = []

      try:
        if str(type(ques)) == "<class 'list'>":

          print("Inside list")

          for i in ques:
            req_ques = {
                'question':i,
                'context':context
            }
            print(req_ques)
            sol = { 'ques':i, 'ans':self.qa(req_ques)['answer'] }

            ans.append(sol)

        else:
          req_ques = {
                'question':ques,
                'context':context
            }
          sol = {
              'ques':ques,
              'ans':self.qa(req_ques)['answer']
          }
          ans.append(sol)
        return ans      

      except:
        pass
    

In [6]:
model = QNA()

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

In [7]:
question = [["Of the two countries that produce soybeans, which country is clearing rain forest in order to increase production?"]]
            
context = ["""
Between 1991 and 2000, the total area of forest lost in the Amazon rose from 415,000 to 587,000 square kilometres (160,000 to 227,000 sq mi), 
with most of the lost forest becoming pasture for cattle. Seventy percent of formerly forested land in the Amazon, and 91% of land deforested 
since 1970, is used for livestock pasture. Currently, Brazil is the second-largest global producer of soybeans after the United States. 
New research however, conducted by Leydimere Oliveira et al., has shown that the more rainforest is logged in the Amazon, the less 
precipitation reaches the area and so the lower the yield per hectare becomes. So despite the popular perception, there has been no 
economical advantage for Brazil from logging rainforest zones and converting these to pastoral fields."""]

In [8]:
cleansed = []
for each in context:
  cleansed.append(model.cleaning_text(each))

In [9]:
ans = []
for i,j in zip(cleansed,question):
  ans.append(model.predict_ans(i,j))

Inside list
{'question': 'Of the two countries that produce soybeans, which country is clearing rain forest in order to increase production?', 'context': 'Between 1991 and 2000, the total area of forest lost in the Amazon rose from 415,000 to 587,000 square kilometres (160,000 to 227,000 sq mi),  with most of the lost forest becoming pasture for cattle. Seventy percent of formerly forested land in the Amazon, and 91% of land deforested  since 1970, is used for livestock pasture. Currently, Brazil is the second-largest global producer of soybeans after the United States.  New research however, conducted by Leydimere Oliveira et al., has shown that the more rainforest is logged in the Amazon, the less  precipitation reaches the area and so the lower the yield per hectare becomes. So despite the popular perception, there has been no  economical advantage for Brazil from logging rainforest zones and converting these to pastoral fields.'}


In [10]:
print("Question: {}".format(ans[0][0]['ques']))
print("Answer: {}".format(ans[0][0]['ans']))

Question: Of the two countries that produce soybeans, which country is clearing rain forest in order to increase production?
Answer: Brazil


In [11]:
model.save_model()